### Setup

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
base_site = "https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda"

In [3]:
response = requests.get(base_site, headers=headers)
response

<Response [200]>

In [4]:
html = response.content

### Choosing a parser

In [5]:
soup = BeautifulSoup(html, 'lxml')

### Obtain the links of announcements

In [6]:
links = soup.find_all(attrs={'data-lurker-detail': 'list_id'})
links_announce = [link["href"] for link in links]
links_announce

['https://pb.olx.com.br/paraiba/imoveis/lancamento-no-bessa-01-02-quartos-com-area-de-lazer-737175447',
 'https://pb.olx.com.br/paraiba/imoveis/excelente-apartamento-no-cristo-735994003',
 'https://pb.olx.com.br/paraiba/imoveis/excelente-apartamento-para-vender-no-cabo-branco-otima-localizacao-737174474',
 'https://pb.olx.com.br/paraiba/imoveis/lindo-apartamento-localizado-no-melhor-do-cabo-branco-no-concept-737173527',
 'https://pb.olx.com.br/paraiba/imoveis/apartamento-em-intermares-01-02-quartos-c-area-de-lazer-na-cobertura-737172545',
 'https://pb.olx.com.br/paraiba/imoveis/excelentes-apartamentos-de-02-e-03-quartos-otima-localizacao-proximo-a-unipe-737171563',
 'https://pb.olx.com.br/paraiba/imoveis/apartamento-no-bessa-02-quartos-excelente-localizacao-proximo-a-praia-737171295',
 'https://pb.olx.com.br/paraiba/imoveis/oportunidade-de-apartamento-03-quartos-com-area-de-lazer-completa-com-02-vagas-737171088',
 'https://pb.olx.com.br/paraiba/imoveis/oportunidade-de-apartamento-pront

### Go inside announce and get data

In [13]:
prices = []
addresses = []

In [14]:
print("Get data inside the announce page")
for link_announce in links_announce:
    announce = requests.get(link_announce, headers=headers)
    if announce.status_code != 200:
        print('Status code {0}: Skipping URL #{1}'.format(announce.status_code, link_announce))
        continue
    announce_html = announce.content
    announce_soup = BeautifulSoup(announce_html, 'lxml')

    price = announce_soup.find("h2").text
    address_check = announce_soup.find("dt", string="Logradouro") or announce_soup.find("dt", string="CEP")
    address = address_check.next_sibling.text
    prices.append(price)
    addresses.append(address)

print("Ok. Announcements on the page: {0}".format(len(prices)))

Get data inside the announce page
Ok. Announcements on the page: 50


### Get page links

In [15]:
pages_links = soup.find_all(attrs={'data-lurker-detail': 'pagination_item'})
pages = [page["href"] for page in pages_links]
pages

['https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=2',
 'https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=3',
 'https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=4',
 'https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=5',
 'https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=6',
 'https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=7',
 'https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=8',
 'https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=9',
 'https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=10',
 'https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=11',
 'https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=12',
 'https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=13',
 'https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=14',
 'https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=15']

### Repeat process inside the pages

In [16]:
for page in pages:
    # Normal setup flow
    print("Get page: {0}".format(page))
    response_page = requests.get(page, headers=headers)
    if response_page.status_code != 200:
        print('Status code {0}: Skipping URL #{1}'.format(response_page.status_code, page))
        continue
    html_page = response_page.content
    soup_page = BeautifulSoup(html_page, 'lxml')
    
    # Get announce link inside the pages
    
    links_page = soup_page.find_all(attrs={'data-lurker-detail': 'list_id'})
    links_announce_page = [link_page["href"] for link_page in links_page]
    
    print("Get data inside the announce page")
    for link_announce_page in links_announce_page:
        announce_page = requests.get(link_announce_page, headers=headers)
        if announce_page.status_code != 200:
            print('Status code {0}: Skipping URL #{1}'.format(announce_page.status_code, link_announce_page))
            continue
        announce_html_page = announce_page.content
        announce_soup_page = BeautifulSoup(announce_html_page, 'lxml')

        price_page = announce_soup_page.find("h2").text
        address_check_page = announce_soup_page.find("dt", string="Logradouro") or announce_soup_page.find("dt", string="CEP")
        address_page = address_check_page.next_sibling.text
        prices.append(price_page)
        addresses.append(address_page)
    print("Ok. Announcements: {0}".format(len(prices)))

Get page: https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=2
Get data inside the announce page
Ok. Announcements: 100
Get page: https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=3
Get data inside the announce page
Ok. Announcements: 150
Get page: https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=4
Get data inside the announce page
Ok. Announcements: 200
Get page: https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=5
Get data inside the announce page
Ok. Announcements: 250
Get page: https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=6
Get data inside the announce page
Ok. Announcements: 300
Get page: https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=7
Get data inside the announce page
Ok. Announcements: 350
Get page: https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=8
Get data inside the announce page
Ok. Announcements: 400
Get page: https://pb.olx.com.br/paraiba/joao-pessoa/imoveis/venda?o=9
Get data inside the announce page
Ok. Ann

### Check generated data

In [17]:
import pandas as pd

In [19]:
imoveis_dt = pd.DataFrame()

imoveis_dt["prices"] = prices
imoveis_dt["address"] = addresses

pd.set_option("max_rows", None)

imoveis_dt

,prices,address
0,R$ 159.900,58036000
1,R$ 145.000,Rua Universitário Rogério Benevides
2,R$ 350.000,Avenida Presidente Epitácio Pessoa - de 4200 a...
3,R$ 420.000,Avenida Presidente Epitácio Pessoa - de 4200 a...
4,R$ 109.900,Rua Fernando Luiz Henriques dos Santos - até 1...
5,R$ 159.000,58031380
6,R$ 169.900,Rua Fernando Luiz Henriques dos Santos - até 1...
7,R$ 270.000,58036000
8,R$ 185.000,58036000
9,R$ 111.000,Rua Manuel de Paula Magalhães


In [20]:
# TODO: GEOCODING ADDRESSES